In [1]:
import numpy as np
from scipy.linalg import inv
from parser import parse_model_from_txt

# Simplex

In [5]:
class Simplex:
    """
        Implementa o algoritmo Simplex de duas fases para resolver problemas de programação linear.

        A classe é projetada para receber um modelo de PL já processado por um parser,
        transformar o problema para a forma padrão, aplicar a Fase 1 (para encontrar uma
        solução básica factível) e a Fase 2 (para encontrar a solução ótima), e então
        retornar a solução em termos das variáveis originais.

        **Args:**
            - *c_parser_vars* (**np.array**): Vetor de custos para as variáveis já transformadas (não-negativas) pelo parser.
            - *A_parser_vars* (**np.array**): Matriz de coeficientes das restrições para as variáveis transformadas.
            - *b_orig* (**np.array**): Vetor de termos independentes (lado direito) das restrições.
            - *signs* (**list**): Lista de strings com os sinais de cada restrição ('<=', '>=', '=').
            - *was_min* (**bool**): True se o problema original era de minimização.
            - *interpretation_info* (**dict**): Dicionário com informações para mapear a solução final de volta às variáveis originais.
            - *m* (**int**): Número de restrições.
            - *n_parser_vars* (**int**): Número de variáveis após o parser (antes de adicionar folga/artificiais).
            - *solution* (**dict**): Armazena o dicionário com o resultado final da otimização.
            - *status* (**str**): Armazena a string de status final da otimização.
            - *tableau* (**np.array**): O tableau do Simplex.
            - *basic_vars* (**list**): Lista de índices das variáveis que estão na base.
            - *num_vars* (**int**): Número total de variáveis no tableau (incluindo folga, excesso e artificiais).
            - *c_phase1* (**np.array**): Vetor de custos para o problema da Fase 1.
            - *artificial_indices* (**list**): Lista de índices das colunas correspondentes às variáveis artificiais no tableau.
    """
    def __init__(self, c_from_parser, A_from_parser, b, signs, was_min=False, interpretation_info=None):
        self.c_parser_vars = np.array(c_from_parser, dtype=float)
        self.A_parser_vars = np.array(A_from_parser, dtype=float)
        self.b_orig = np.array(b, dtype=float)
        self.signs = signs
        self.was_min = was_min
        self.interpretation_info = interpretation_info
        self.m, self.n_parser_vars = self.A_parser_vars.shape

        # Atributos preenchidos por _prepare_problem
        self.A = None
        self.c = None
        self.b = None
        self.num_vars = None

    def _prepare_problem(self):
        """Prepara os dados básicos (A, b, c) para a forma padrão."""
        self.b = np.copy(self.b_orig)
        temp_A = np.copy(self.A_parser_vars)
        
        for i in range(self.m):
            if self.b[i] < 0:
                self.b[i] *= -1
                temp_A[i, :] *= -1
                if self.signs[i] == '<=': self.signs[i] = '>='
                elif self.signs[i] == '>=': self.signs[i] = '<='

        num_slack_vars = sum(1 for sign in self.signs if sign != '=')
        self.num_vars = self.n_parser_vars + num_slack_vars
        
        self.A = np.zeros((self.m, self.num_vars))
        self.A[:, :self.n_parser_vars] = temp_A
        self.c = np.zeros(self.num_vars)
        self.c[:self.n_parser_vars] = self.c_parser_vars

        slack_ptr = self.n_parser_vars
        for i, sign in enumerate(self.signs):
            if sign == '<=':
                self.A[i, slack_ptr] = 1.0
                slack_ptr += 1
            elif sign == '>=':
                self.A[i, slack_ptr] = -1.0
                slack_ptr += 1

    # --------------------------------------------------------------------------
    # FLUXO DE SOLUÇÃO PRINCIPAL
    # --------------------------------------------------------------------------
    def solve(self, method='revised'):
        """
        Ponto de entrada principal. Despacha para o método de solução escolhido.
        """
        self._prepare_problem()
        
        if method == 'revised':
            result = self._solve_revised()
        elif method == 'tabular':
            result = self._solve_tabular()
        else:
            raise ValueError("Método inválido. Escolha 'tabular' ou 'revised'.")
            
        return self._format_final_solution(result)

    # --------------------------------------------------------------------------
    # LÓGICA PARA O MÉTODO REVISADO
    # --------------------------------------------------------------------------
    def _solve_revised(self):
        """Orquestra a solução usando o método Simplex Revisado."""
        # 1. Executa a Fase 1 para obter uma base factível
        phase1_result = self._run_phase1_revised()
        if phase1_result.get('status') != 'feasible':
            return phase1_result
        
        initial_base_indices = phase1_result['base']
        
        # 2. Executa a Fase 2 com a base encontrada
        return self._revised_simplex_engine(self.A, self.b, self.c, initial_base_indices)

    def _run_phase1_revised(self):
        """Executa a Fase 1 para o método revisado."""
        artificial_rows = {i for i, sign in enumerate(self.signs) if sign in ['>=', '=']}
        if not artificial_rows:
            # Base trivial com vars de folga (se houver)
            slack_indices = [self.n_parser_vars + i for i, s in enumerate(self.signs) if s != '=']
            return {'status': 'feasible', 'base': slack_indices}

        num_artificial = len(artificial_rows)
        A_phase1 = np.hstack([self.A, np.zeros((self.m, num_artificial))])
        c_phase1 = np.zeros(self.A.shape[1] + num_artificial)
        c_phase1[self.A.shape[1]:] = -1.0
        
        initial_base_phase1 = [-1] * self.m
        art_ptr = self.A.shape[1]
        slack_ptr = self.n_parser_vars
        for i in range(self.m):
            if i in artificial_rows:
                A_phase1[i, art_ptr] = 1.0
                initial_base_phase1[i] = art_ptr
                art_ptr += 1
            else: # Restrição <=
                initial_base_phase1[i] = slack_ptr
                slack_ptr += 1
        
        result_phase1 = self._revised_simplex_engine(A_phase1, self.b, c_phase1, initial_base_phase1)

        if result_phase1.get('status') != 'optimal' or abs(result_phase1.get('value', 0)) > 1e-9:
            return {'status': 'infeasible'}

        final_base_phase1 = result_phase1['final_basis_indices']
        
        # Se alguma variável artificial está na base, ela deve ser expulsa
        if any(b >= self.A.shape[1] for b in final_base_phase1):
             return {'status': 'error_redundant_constraint', 'message': 'Não foi possível expulsar as variáveis artificiais da base. O modelo pode ter restrições redundantes.'}

        return {'status': 'feasible', 'base': final_base_phase1}

    def _revised_simplex_engine(self, A, b, c, initial_basic_indices):
        """Motor do Simplex Revisado."""
        basic_indices = np.array(initial_basic_indices, dtype=int)
        num_vars = A.shape[1]
        
        for _ in range(self.m * num_vars * 2): # Limite de iterações
            B = A[:, basic_indices]
            try:
                B_inv = inv(B)
            except np.linalg.LinAlgError:
                return {'status': 'error_singular_matrix'}
            
            non_basic_indices = np.setdiff1d(np.arange(num_vars), basic_indices)
            c_b = c[basic_indices]
            x_b = B_inv @ b
            y = c_b @ B_inv
            cj_zj = c[non_basic_indices] - y @ A[:, non_basic_indices]

            if np.all(cj_zj <= 1e-9):
                sol = np.zeros(num_vars)
                sol[basic_indices] = x_b
                return {'status': 'optimal', 'solution': sol, 'value': c_b @ x_b,
                        'is_degenerate': np.any(np.isclose(x_b, 0)),
                        'has_multiple_solutions': np.any(np.isclose(cj_zj, 0)),
                        'final_basis_indices': basic_indices, 'final_B_inv': B_inv}
            
            entering_idx = non_basic_indices[np.argmax(cj_zj)]
            d = B_inv @ A[:, entering_idx]
            
            if np.all(d <= 1e-9): return {'status': 'unbounded'}
            
            ratios = np.array([x_b[i] / d[i] if d[i] > 1e-9 else np.inf for i in range(self.m)])
            leaving_row = np.argmin(ratios)
            basic_indices[leaving_row] = entering_idx
        
        return {'status': 'max_iterations_reached'}

    # --------------------------------------------------------------------------
    # LÓGICA PARA O MÉTODO TABULAR
    # --------------------------------------------------------------------------
    def _solve_tabular(self):
        """Orquestra a solução usando o método Simplex Tabular."""
        # 1. Construir e resolver o tableau da Fase 1
        tableau, basic_indices, artificial_indices, status = self._build_and_run_phase1_tabular()
        if status != 'optimal':
            return {'status': status} # Retorna 'infeasible' ou outro erro

        # 2. Preparar e resolver o tableau da Fase 2
        tableau, basic_indices = self._prepare_phase2_tableau(tableau, basic_indices, artificial_indices)
        return self._tabular_simplex_engine(tableau, basic_indices, self.c)
        
    def _build_and_run_phase1_tabular(self):
        """Cria e resolve o tableau da Fase 1."""
        num_artificial = sum(1 for sign in self.signs if sign in ['>=', '='])
        num_slack = self.num_vars - self.n_parser_vars
        
        tableau_width = self.n_parser_vars + num_slack + num_artificial + 1
        tableau = np.zeros((self.m, tableau_width))
        tableau[:, :self.n_parser_vars] = self.A[:, :self.n_parser_vars]
        tableau[:, -1] = self.b
        
        basic_indices = [-1] * self.m
        artificial_indices = []
        
        c_phase1 = np.zeros(tableau_width - 1)
        
        slack_ptr = self.n_parser_vars
        art_ptr = self.n_parser_vars + num_slack
        
        for i in range(self.m):
            sign = self.signs[i]
            if sign == '<=':
                tableau[i, slack_ptr] = 1.0
                basic_indices[i] = slack_ptr
                slack_ptr += 1
            else: # >= ou =
                if sign == '>=':
                    tableau[i, slack_ptr] = -1.0
                    slack_ptr += 1
                tableau[i, art_ptr] = 1.0
                basic_indices[i] = art_ptr
                artificial_indices.append(art_ptr)
                c_phase1[art_ptr] = -1.0 # max -w
                art_ptr += 1

        result = self._tabular_simplex_engine(tableau, basic_indices, c_phase1)
        
        if result.get('status') != 'optimal' or abs(result.get('value', 0)) > 1e-9:
            return None, None, None, 'infeasible'
        
        return result['tableau'], result['final_basis_indices'], artificial_indices, 'optimal'

    def _prepare_phase2_tableau(self, tableau, basic_indices, artificial_indices):
        """Modifica o tableau da Fase 1 para a Fase 2."""
        # Remove colunas artificiais
        cols_to_keep = [i for i in range(tableau.shape[1] - 1) if i not in artificial_indices]
        tableau = tableau[:, cols_to_keep + [tableau.shape[1] - 1]]
        
        # Atualiza os índices da base
        map_old_to_new = {old: new for new, old in enumerate(cols_to_keep)}
        new_basic_indices = [map_old_to_new[b] for b in basic_indices]
        
        return tableau, new_basic_indices

    def _tabular_simplex_engine(self, tableau_init, basic_indices_init, c_original):
        """
        Motor do Simplex Tabular.
        Versão corrigida que não modifica o vetor de custos original.
        """
        tableau = np.copy(tableau_init)
        basic_indices = list(basic_indices_init)
        num_vars = tableau.shape[1] - 1
        
        # Limite de iterações para evitar loops infinitos
        for _ in range(self.m * num_vars * 2):
            # 1. Calcular os custos reduzidos (cj - zj) usando o vetor de custos original
            cb = c_original[basic_indices]
            zj = cb @ tableau[:, :-1]
            cj_zj = c_original - zj

            # Zera os custos reduzidos das variáveis básicas por precisão numérica
            cj_zj[basic_indices] = 0

            # 2. Verificar a condição de otimalidade
            if np.all(cj_zj <= 1e-9):
                solution = np.zeros(num_vars)
                for i, idx in enumerate(basic_indices):
                    solution[idx] = tableau[i, -1]
                
                # Checa por múltiplas soluções nas variáveis NÃO básicas
                non_basic_indices = np.setdiff1d(np.arange(num_vars), basic_indices)
                has_multiple_solutions = np.any(np.isclose(cj_zj[non_basic_indices], 0))

                return {
                    'status': 'optimal',
                    'solution': solution,
                    'value': cb @ solution[basic_indices],
                    'is_degenerate': np.any(np.isclose(tableau[:, -1], 0)),
                    'has_multiple_solutions': has_multiple_solutions,
                    'tableau': tableau,
                    'final_basis_indices': basic_indices
                }

            # 3. Encontrar a variável para entrar na base
            entering_col = np.argmax(cj_zj)
            
            # 4. Verificar se a solução é ilimitada
            if np.all(tableau[:, entering_col] <= 1e-9):
                return {'status': 'unbounded'}

            # 5. Encontrar a variável para sair da base (teste da razão)
            ratios = np.array([tableau[i, -1] / tableau[i, entering_col] if tableau[i, entering_col] > 1e-9 else np.inf for i in range(self.m)])
            leaving_row = np.argmin(ratios)
            
            # 6. Realizar o pivoteamento
            pivot_element = tableau[leaving_row, entering_col]
            if abs(pivot_element) < 1e-9:
                return {'status': 'error_numerical_instability'}
            
            tableau[leaving_row, :] /= pivot_element
            for i in range(self.m):
                if i != leaving_row:
                    tableau[i, :] -= tableau[i, entering_col] * tableau[leaving_row, :]
            
            basic_indices[leaving_row] = entering_col

        return {'status': 'max_iterations_reached'}

    # --------------------------------------------------------------------------
    # PÓS-PROCESSAMENTO
    # --------------------------------------------------------------------------
    def _format_final_solution(self, result):
        """Formata o dicionário de resultado final para o utilizador."""
        if result.get('status') != 'optimal':
            return result
        
        sol_vector = result['solution']
        final_sol = {}
        for var, info in self.interpretation_info['simplex_vars_map'].items():
            cols = info['cols_parser']
            if info['type'] == 'free':
                final_sol[var] = sol_vector[cols[0]] - sol_vector[cols[1]]
            else:
                final_sol[var] = sol_vector[cols[0]] * info['mult']
        
        ordered_sol = [final_sol[v] for v in self.interpretation_info['sorted_original_vars']]
        
        # Recalcula valor final com base nos coeficientes originais
        final_value = self.c_parser_vars @ ordered_sol
        
        return {'status': 'optimal', 'solution': ordered_sol,
                'value': -final_value if self.was_min else final_value,
                'is_degenerate': result.get('is_degenerate', False),
                'has_multiple_solutions': result.get('has_multiple_solutions', False)}


### Análise dos Parâmetros de Retorno da Função `parse_model_from_txt`

A função `parse_model_from_txt` retorna um único dicionário que encapsula todo o problema de Programação Linear (PL) em um formato estruturado, pronto para ser processado pelo algoritmo Simplex. Abaixo, detalhamos cada chave deste dicionário.

---

#### chave: `c`
* **Tipo**: `list` de `float`
* **Descrição**: Representa o vetor de custos, ou seja, os coeficientes da função objetivo.
* **Detalhes**:
    * Este vetor já está ajustado para o formato de maximização que o Simplex utiliza. Se o problema original for `min z`, a função o converte para `max -z`, e os coeficientes em `c` serão os do problema original multiplicados por -1.
    * Os coeficientes são mapeados para as variáveis do Simplex, que são todas não-negativas. Por exemplo, se a variável original `x1` for livre (transformada em `x1_p - x1_n`), um termo `5x1` no objetivo resultará em `5` e `-5` nas posições correspondentes de `c` para `x1_p` e `x1_n`.

---

#### chave: `A`
* **Tipo**: `list` de `list` de `float`
* **Descrição**: Representa a matriz `A` de coeficientes das restrições.
* **Detalhes**:
    * Cada lista interna corresponde a uma restrição (linha da matriz).
    * As colunas da matriz correspondem às variáveis do Simplex (ex: `x1_p`, `x1_n`, `x2`, ...) na ordem definida internamente pelo parser.

---

#### chave: `b`
* **Tipo**: `list` de `float`
* **Descrição**: Contém os termos independentes, ou seja, os valores do lado direito (RHS - Right-Hand Side) de cada restrição.
* **Detalhes**:
    * A ordem dos valores em `b` corresponde diretamente à ordem das restrições (linhas) na matriz `A`.

---

#### chave: `signs`
* **Tipo**: `list` de `str`
* **Descrição**: Armazena os sinais de relação (`<=`, `>=`, `=`) para cada uma das restrições.
* **Detalhes**:
    * A ordem dos sinais corresponde à ordem das restrições em `A` e `b`.

---

#### chave: `was_min`
* **Tipo**: `bool`
* **Descrição**: Um indicador booleano que "lembra" se o problema original era de minimização.
* **Detalhes**:
    * `True`: O problema original era de minimização.
    * `False`: O problema original era de maximização.
    * **Finalidade**: O solver Simplex é implementado para sempre maximizar. Esta flag é usada ao final do processo para determinar se o valor ótimo encontrado deve ser multiplicado por -1 para fornecer a resposta correta para o problema de minimização original.

---

#### chave: `interpretation_info`
* **Tipo**: `dict`
* **Descrição**: Este dicionário é fundamental. Ele contém todo o "mapa" necessário para traduzir a solução final, que está em termos de variáveis do Simplex, de volta para os termos das variáveis originais do problema.

##### Sub-chave: `sorted_original_vars`
* **Tipo**: `list` de `str`
* **Descrição**: Uma lista com os nomes das variáveis originais do problema (ex: `['x1', 'x2', 'x3']`), ordenadas para garantir uma apresentação de resultados consistente.

##### Sub-chave: `simplex_var_column_names`
* **Tipo**: `list` de `str`
* **Descrição**: Contém os nomes das variáveis que o Simplex efetivamente utiliza. Por exemplo, se `x1` for uma variável livre, esta lista poderia conter `['x1_p', 'x1_n', 'x2']`.

##### Sub-chave: `simplex_vars_map`
* **Tipo**: `dict`
* **Descrição**: O núcleo da tradução. É um dicionário que mapeia o nome de cada variável **original** para um outro dicionário que detalha sua transformação.
    * **`type` (`str`)**: O tipo da variável original. Pode ser:
        * `'non_negative'`: Para variáveis $x_i \ge 0$.
        * `'free'`: Para variáveis livres.
        * `'negative'`: Para variáveis $x_i \le 0$.
    * **`cols_parser` (`list[int]`)**: Os índices das colunas que a variável original ocupa na matriz `A` e no vetor `c`. Uma variável não-negativa terá um único índice (ex: `[0]`), enquanto uma livre terá dois (ex: `[0, 1]`).
    * **`mult` (`int`)**: Um multiplicador usado na transformação dos coeficientes. É `1` para variáveis não-negativas/livres e `-1` para variáveis negativas (na transformação `x_orig = -x_prime`).
    * **`simplex_names` (`list[str]`)**: Os nomes formais dados às variáveis do Simplex que representam a variável original (ex: `['x1_p', 'x1_n']`).

# Converter Modelo no TXT

In [6]:
model = parse_model_from_txt(r'./modelos/teste.txt')
print(model)


{'c': [10.0, 8.0], 'A': [[2.0, 1.0], [1.0, 1.0], [3.0, 2.0]], 'b': [8.0, 5.0, 13.0], 'signs': ['<=', '<=', '<='], 'was_min': False, 'interpretation_info': {'sorted_original_vars': ['x1', 'x2'], 'simplex_vars_map': {'x1': {'type': 'non_negative', 'cols_parser': [0], 'mult': 1, 'simplex_names': ['x1']}, 'x2': {'type': 'non_negative', 'cols_parser': [1], 'mult': 1, 'simplex_names': ['x2']}}, 'simplex_var_column_names': ['x1', 'x2']}}


In [7]:
simplex = Simplex(model['c'], model['A'], 
                  model['b'], model['signs'], was_min=model['was_min'], 
                  interpretation_info=model['interpretation_info'])

result = simplex.solve("tabular")
print(result)
result = simplex.solve("revised")
print(result)

{'status': 'optimal', 'solution': [3.0, 2.0], 'value': 46.0, 'is_degenerate': True, 'has_multiple_solutions': False}
{'status': 'optimal', 'solution': [3.0, 2.0], 'value': 46.0, 'is_degenerate': True, 'has_multiple_solutions': False}


# Rodar todos os modelos

In [8]:
import os
import glob

In [9]:
def run_all_models(model_directory='modelos'):
    """
    Lê todos os arquivos de modelo .txt de um diretório e seus subdiretórios,
    resolve cada um usando os métodos tabular e revisado, e imprime os resultados.

    Args:
        model_directory (str): O caminho para a pasta contendo os arquivos de modelo.
    """
    # A função glob encontra todos os arquivos que correspondem ao padrão
    model_files = glob.glob(os.path.join(model_directory, '**', '*.txt'), recursive=True)
    
    if not model_files:
        print(f"Nenhum arquivo de modelo .txt encontrado no diretório '{model_directory}'.")
        return
        
    for filepath in sorted(model_files):
        print(f"=====================================================")
        print(f"Executando modelo: {filepath}")
        print(f"=====================================================")
        
        try:
            model_data = parse_model_from_txt(filepath)
        except Exception as e:
            print(f"Erro ao fazer o parse do arquivo {filepath}: {e}\n")
            continue

        # Executa com o método Revisado
        print("\n--- Método: Revisado ---")
        try:
            # É importante criar uma nova instância para cada execução
            # para garantir que o estado não seja compartilhado.
            simplex_revised = Simplex(
                c_from_parser=model_data['c'],
                A_from_parser=model_data['A'],
                b=model_data['b'],
                signs=model_data['signs'],
                was_min=model_data['was_min'],
                interpretation_info=model_data['interpretation_info']
            )
            solution_revised = simplex_revised.solve(method='revised')
            print(solution_revised)
        except Exception as e:
            print(f"Ocorreu um erro durante a execução do método revisado: {e}")

        # Executa com o método Tabular
        print("\n--- Método: Tabular ---")
        try:
            simplex_tabular = Simplex(
                c_from_parser=model_data['c'],
                A_from_parser=model_data['A'],
                b=model_data['b'],
                signs=model_data['signs'],
                was_min=model_data['was_min'],
                interpretation_info=model_data['interpretation_info']
            )
            solution_tabular = simplex_tabular.solve(method='tabular')
            print(solution_tabular)
        except Exception as e:
            print(f"Ocorreu um erro durante a execução do método tabular: {e}")
            
        print("\n")

In [10]:
# --- Execução Principal ---
if __name__ == '__main__':
    # Esta função irá procurar por uma pasta chamada 'modelos' no mesmo
    # diretório onde o script for executado.
    run_all_models(model_directory='modelos')

Executando modelo: modelos\degenerada_1.txt

--- Método: Revisado ---
{'status': 'optimal', 'solution': [1.0, 3.0], 'value': 9.0, 'is_degenerate': True, 'has_multiple_solutions': False}

--- Método: Tabular ---
{'status': 'optimal', 'solution': [1.0, 3.0], 'value': 9.0, 'is_degenerate': True, 'has_multiple_solutions': False}


Executando modelo: modelos\ilimitada_1.txt

--- Método: Revisado ---
{'status': 'unbounded'}

--- Método: Tabular ---
{'status': 'unbounded'}


Executando modelo: modelos\infactivel_1.txt

--- Método: Revisado ---
{'status': 'infeasible'}

--- Método: Tabular ---
{'status': 'infeasible'}


Executando modelo: modelos\infactivel_2.txt

--- Método: Revisado ---
{'status': 'infeasible'}

--- Método: Tabular ---
{'status': 'infeasible'}


Executando modelo: modelos\modelo_online1.txt

--- Método: Revisado ---
{'status': 'optimal', 'solution': [5.0, 4.0, 0.0], 'value': 13.0, 'is_degenerate': True, 'has_multiple_solutions': False}

--- Método: Tabular ---
{'status': 'op